Mounting drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


A Couple Installs

In [2]:
%pip install -q tfds-nightly tensorflow matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 40.1 MB/s eta 0:00:00


Libraries to Import

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Datasets

Reading the datasets

In [4]:
ds = tfds.load('plant_village', split='train', shuffle_files=True, as_supervised=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Shuffling /root/tensorflow_datasets/plant_village/1.0.2.incompleteXMIHGB/plant_village-train.tfrecord*...:   0…

Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.2. Subsequent calls will reuse this data.
<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


Getting DS size and splitting into train and test

In [5]:
def get_ds_size(ds): 
  ds_size = 0
  for elem in ds: 
    ds_size += 1
  return ds_size

In [6]:
def partition_ds(ds, ds_size, train_rate, shuffle=True, shuffle_size=10000): 
  assert 0 < train_rate < 1
  if shuffle: 
    ds.shuffle(shuffle_size, seed = 123)
  
  # Determine size of train ds
  train_size = int(train_rate * ds_size)

  # Fetch and return both fragmented ds
  train_ds = ds.take(train_size)    
  test_ds = ds.skip(train_size)

  return train_ds, test_ds

In [14]:
ds_size = get_ds_size(ds)
train_val_ds, test_ds = partition_ds(ds, ds_size, 0.8)
train_val_size = get_ds_size(train_val_ds)
train_ds, val_ds = partition_ds(train_val_ds, train_val_size, 0.8, shuffle_size=train_val_size//5)

In [15]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

def prepare_train_ds(train_ds): 
  # Get shuffle size 
  train_shuffle_size = get_ds_size(train_ds)//5

  train_ds = train_ds.map(
      normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
  train_ds = train_ds.cache()
  train_ds = train_ds.shuffle(train_shuffle_size)
  train_ds = train_ds.batch(32)
  train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
  return train_ds

def prepare_test_ds(test_ds): 
  test_ds = test_ds.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
  test_ds = test_ds.batch(32)
  test_ds = test_ds.cache()
  test_ds = test_ds.prefetch(tf.data.AUTOTUNE)
  return test_ds

Two datasets fully prepared. Fixed RAM session crash issue. 

In [16]:
train_ds = prepare_train_ds(train_ds)
test_ds = prepare_test_ds(test_ds)

In [17]:
print(train_ds)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [19]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 256, 256, 3)
(32,)


## Model

In [20]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(256, 256)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])

In [21]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

RAM Session crashing here as well.

In [ ]:
model.fit(
    train_ds,
    epochs=5,
    validation_data=test_ds,
)

Epoch 1/5
